# 🚀 Lab DataOps: Governança e Qualidade de Dados com PySpark e Great Expectations

## Objetivos do Laboratório

Neste laboratório prático, você irá:
- Implementar **testes de qualidade de dados** automatizados com **Great Expectations**
- Aplicar as **6 dimensões da qualidade** na prática
- Criar um **pipeline DataOps** com validações profissionais
- Simular cenários reais de **governança de dados**
- Gerar **relatórios de qualidade** automatizados

### Conceitos Aplicados
- **DataOps**: Automação e monitoramento contínuo
- **Governança**: Regras, políticas e responsabilidades
- **Qualidade**: Acurácia, Completude, Consistência, Pontualidade, Unicidade e Validade
- **Great Expectations**: Framework profissional para validação de dados

## 1. Configuração do Ambiente PySpark e Great Expectations

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
from datetime import datetime, timedelta
import os

# Great Expectations imports
try:
    import great_expectations as gx
    print(f"✅ Great Expectations versão: {gx.__version__}")
except ImportError:
    print("❌ Great Expectations não encontrado. Instalando...")
    !pip install great-expectations==0.18.8 sqlalchemy==1.4.46
    import great_expectations as gx
    print(f"✅ Great Expectations instalado: {gx.__version__}")

# Inicializar Spark Session
spark = SparkSession.builder \
    .appName("DataOps_Governanca_Lab_GX") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"✅ Spark Session iniciada: {spark.version}")
print(f"📊 Contexto: {spark.sparkContext.appName}")
print(f"🎯 Great Expectations versão: {gx.__version__}")

## 2. Criação de Dados de Exemplo com PySpark

In [ ]:
# Dados simulados com problemas de qualidade intencionais
dados_clientes = [
    (1, "João Silva", "joao@email.com", "11999887766", "2023-01-15", "Ativo", 25, "SP"),
    (2, "Maria Santos", "maria.santos@gmail.com", "11888776655", "2023-02-20", "Ativo", 32, "RJ"),
    (3, "Pedro", "pedro@invalid", "119999", "2023-03-10", "Inativo", 150, "MG"),  # Problemas: email inválido, telefone incompleto, idade impossível
    (4, "Ana Costa", "ana@email.com", "11777665544", "2023-04-05", "Ativo", 28, "SP"),
    (1, "João Silva", "joao@email.com", "11999887766", "2023-01-15", "Ativo", 25, "SP"),  # Duplicata
    (5, "", "carlos@email.com", "11666554433", "2023-05-12", "Pendente", None, "RS"),  # Nome vazio, idade nula
    (6, "Lucia Oliveira", "lucia@email.com", "11555443322", "2022-12-01", "Ativo", 45, "BA"),
    (7, "Roberto Lima", "roberto@email.com", "11444332211", "2023-06-18", "Cancelado", 38, "PR"),  # Status não padrão
    (8, "Fernanda", None, "11333221100", "2023-07-22", "Ativo", 29, "SC"),  # Email nulo
    (9, "Marcos Pereira", "marcos@email.com", "11222110099", "2023-08-30", "Ativo", -5, "GO")  # Idade negativa
]

# Schema definido (Governança: Padrões de Dados)
schema_clientes = StructType([
    StructField("id_cliente", IntegerType(), False),
    StructField("nome", StringType(), True),
    StructField("email", StringType(), True),
    StructField("telefone", StringType(), True),
    StructField("data_cadastro", StringType(), True),
    StructField("status", StringType(), True),
    StructField("idade", IntegerType(), True),
    StructField("estado", StringType(), True)
])

df_clientes_spark = spark.createDataFrame(dados_clientes, schema_clientes)
print("📋 Dataset de clientes criado com problemas de qualidade intencionais")
df_clientes_spark.show()

# Converter para pandas para Great Expectations (evita problemas de serialização)
df_clientes_pandas = df_clientes_spark.toPandas()
print(f"📊 Dataset convertido para pandas: {len(df_clientes_pandas)} registros")

## 3. Configuração do Great Expectations

In [ ]:
# Configurar diretório do Great Expectations
project_dir = "/tmp/gx_lab_dataops"
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

# Inicializar Data Context do Great Expectations
try:
    context = gx.get_context()
    print("📁 Contexto Great Expectations carregado")
except:
    context = gx.get_context(project_root_dir=project_dir)
    print("📁 Novo contexto Great Expectations criado")

print(f"✅ Great Expectations configurado")
print(f"📊 Dataset preparado: {len(df_clientes_pandas)} registros")

## 4. Criação de Expectativas de Qualidade

In [ ]:
# Criar validator usando pandas (evita problemas de serialização do Spark)
try:
    validator = context.sources.pandas_default.read_dataframe(df_clientes_pandas)
    print("✅ Validator criado com sucesso")
except Exception as e:
    print(f"⚠️ Erro: {e}")
    print("🔧 Tentando método alternativo...")
    # Método alternativo mais básico
    validator = gx.validator.validator.Validator(
        execution_engine=gx.execution_engine.PandasExecutionEngine(),
        interactive_evaluation=True
    )
    validator.load_batch_list([gx.core.batch.Batch(data=df_clientes_pandas)])
    print("✅ Validator criado com método alternativo")

## 5. Definindo Expectativas das 6 Dimensões da Qualidade

In [ ]:
print("🎯 CRIANDO EXPECTATIVAS DAS 6 DIMENSÕES DA QUALIDADE")
print("=" * 60)

# 1. COMPLETUDE (Completeness) - Campos não podem ser nulos
print("\n📊 1. COMPLETUDE - Campos obrigatórios")
validator.expect_column_values_to_not_be_null("id_cliente")
validator.expect_column_values_to_not_be_null("nome")
validator.expect_column_values_to_not_be_null("email")
print("   ✅ Expectativas de completude criadas")

# 2. UNICIDADE (Uniqueness) - Valores únicos
print("\n🔑 2. UNICIDADE - Chaves únicas")
validator.expect_column_values_to_be_unique("id_cliente")
validator.expect_column_values_to_be_unique("email")
print("   ✅ Expectativas de unicidade criadas")

# 3. VALIDADE (Validity) - Formatos e domínios
print("\n✅ 3. VALIDADE - Formatos e domínios")
# Email válido
validator.expect_column_values_to_match_regex(
    "email", 
    r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
)
# Idade válida (0-120)
validator.expect_column_values_to_be_between("idade", min_value=0, max_value=120)
# Status válido
validator.expect_column_values_to_be_in_set("status", ["Ativo", "Inativo", "Pendente"])
print("   ✅ Expectativas de validade criadas")

# 4. CONSISTÊNCIA (Consistency) - Regras de negócio
print("\n🔄 4. CONSISTÊNCIA - Regras de negócio")
# Telefone deve ter 11 dígitos
validator.expect_column_value_lengths_to_equal("telefone", 11)
# Data de cadastro não pode ser futura
validator.expect_column_values_to_be_dateutil_parseable("data_cadastro")
print("   ✅ Expectativas de consistência criadas")

# 5. PONTUALIDADE (Timeliness) - Dados atuais
print("\n⏰ 5. PONTUALIDADE - Dados atuais")
# Data de cadastro deve ser dos últimos 2 anos
data_limite = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
validator.expect_column_values_to_be_between(
    "data_cadastro", 
    min_value=data_limite, 
    max_value=datetime.now().strftime("%Y-%m-%d")
)
print("   ✅ Expectativas de pontualidade criadas")

# 6. ACURÁCIA (Accuracy) - Dados corretos
print("\n🎯 6. ACURÁCIA - Dados corretos")
# Nome não pode ser vazio
validator.expect_column_values_to_not_match_regex("nome", r"^\s*$")
# Estado deve ser sigla válida (2 caracteres)
validator.expect_column_value_lengths_to_equal("estado", 2)
print("   ✅ Expectativas de acurácia criadas")

print("\n💾 Todas as expectativas criadas com sucesso!")

## 6. Execução das Validações

In [ ]:
# Executar validações
print("🔍 EXECUTANDO VALIDAÇÕES DE QUALIDADE")
print("=" * 50)

# Validar os dados
validation_result = validator.validate()

# Analisar resultados
print(f"\n📊 RESULTADOS DA VALIDAÇÃO:")
print(f"   Total de expectativas: {validation_result.statistics['evaluated_expectations']}")
print(f"   Expectativas que passaram: {validation_result.statistics['successful_expectations']}")
print(f"   Expectativas que falharam: {validation_result.statistics['unsuccessful_expectations']}")
print(f"   Taxa de sucesso: {validation_result.statistics['success_percent']:.1f}%")

# Status geral
if validation_result.success:
    print("\n✅ VALIDAÇÃO PASSOU - Dados dentro dos padrões de qualidade")
else:
    print("\n❌ VALIDAÇÃO FALHOU - Problemas de qualidade detectados")
    print("\n🚨 PROBLEMAS ENCONTRADOS:")
    
    for result in validation_result.results:
        if not result.success:
            expectation_type = result.expectation_config.expectation_type
            column = result.expectation_config.kwargs.get('column', 'N/A')
            print(f"   ❌ {expectation_type} - Coluna: {column}")
            if 'partial_unexpected_count' in result.result:
                print(f"      Registros com problema: {result.result['partial_unexpected_count']}")

## 7. Correção de Dados com PySpark

In [ ]:
def corrigir_dados_qualidade_spark(df_spark):
    """
    Aplica correções usando PySpark baseadas nos resultados das validações
    """
    print("🔧 APLICANDO CORREÇÕES DE QUALIDADE COM PYSPARK")
    print("=" * 50)
    
    # 1. Remover duplicatas (manter primeira ocorrência)
    df_corrigido = df_spark.dropDuplicates(["id_cliente"])
    print(f"✅ Duplicatas removidas: {df_spark.count() - df_corrigido.count()} registros")
    
    # 2. Corrigir idades inválidas
    df_corrigido = df_corrigido.withColumn(
        "idade",
        when((col("idade") < 0) | (col("idade") > 120), None)
        .otherwise(col("idade"))
    )
    print("✅ Idades inválidas corrigidas")
    
    # 3. Padronizar status
    df_corrigido = df_corrigido.withColumn(
        "status",
        when(col("status") == "Cancelado", "Inativo")
        .otherwise(col("status"))
    )
    print("✅ Status padronizados")
    
    # 4. Remover registros com campos críticos vazios
    df_corrigido = df_corrigido.filter(
        col("nome").isNotNull() & 
        (col("nome") != "") &
        col("email").isNotNull()
    )
    print("✅ Registros com campos críticos vazios removidos")
    
    print(f"\n📊 Registros após correção: {df_corrigido.count()}")
    return df_corrigido

# Aplicar correções
df_clientes_corrigido_spark = corrigir_dados_qualidade_spark(df_clientes_spark)
df_clientes_corrigido_pandas = df_clientes_corrigido_spark.toPandas()

print("\n📋 Dados após correção:")
df_clientes_corrigido_spark.show()

## 8. Re-validação com Dados Corrigidos

In [ ]:
print("🔄 RE-VALIDAÇÃO COM DADOS CORRIGIDOS")
print("=" * 50)

# Criar novo validator com dados corrigidos
try:
    validator_corrigido = context.sources.pandas_default.read_dataframe(df_clientes_corrigido_pandas)
    print("✅ Novo validator criado")
except Exception as e:
    print(f"⚠️ Usando método alternativo: {str(e)[:50]}...")
    validator_corrigido = gx.validator.validator.Validator(
        execution_engine=gx.execution_engine.PandasExecutionEngine(),
        interactive_evaluation=True
    )
    validator_corrigido.load_batch_list([gx.core.batch.Batch(data=df_clientes_corrigido_pandas)])

# Aplicar as mesmas expectativas
print("🎯 Aplicando expectativas aos dados corrigidos...")

# Completude
validator_corrigido.expect_column_values_to_not_be_null("id_cliente")
validator_corrigido.expect_column_values_to_not_be_null("nome")
validator_corrigido.expect_column_values_to_not_be_null("email")

# Unicidade
validator_corrigido.expect_column_values_to_be_unique("id_cliente")
validator_corrigido.expect_column_values_to_be_unique("email")

# Validade
validator_corrigido.expect_column_values_to_match_regex("email", r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$")
validator_corrigido.expect_column_values_to_be_between("idade", min_value=0, max_value=120)
validator_corrigido.expect_column_values_to_be_in_set("status", ["Ativo", "Inativo", "Pendente"])

# Consistência
validator_corrigido.expect_column_value_lengths_to_equal("telefone", 11)
validator_corrigido.expect_column_values_to_be_dateutil_parseable("data_cadastro")

# Pontualidade
data_limite = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
validator_corrigido.expect_column_values_to_be_between("data_cadastro", min_value=data_limite, max_value=datetime.now().strftime("%Y-%m-%d"))

# Acurácia
validator_corrigido.expect_column_values_to_not_match_regex("nome", r"^\s*$")
validator_corrigido.expect_column_value_lengths_to_equal("estado", 2)

# Executar validação
validation_result_corrigido = validator_corrigido.validate()

# Comparar resultados
print("\n📊 COMPARAÇÃO DE RESULTADOS")
print("=" * 40)
print(f"DADOS ORIGINAIS:")
print(f"   Taxa de sucesso: {validation_result.statistics['success_percent']:.1f}%")
print(f"   Expectativas que falharam: {validation_result.statistics['unsuccessful_expectations']}")

print(f"\nDADOS CORRIGIDOS:")
print(f"   Taxa de sucesso: {validation_result_corrigido.statistics['success_percent']:.1f}%")
print(f"   Expectativas que falharam: {validation_result_corrigido.statistics['unsuccessful_expectations']}")

if validation_result_corrigido.success:
    print("\n🎉 SUCESSO! Dados corrigidos passaram em todas as validações")
else:
    print("\n⚠️ Ainda existem problemas nos dados corrigidos")
    
melhoria = validation_result_corrigido.statistics['success_percent'] - validation_result.statistics['success_percent']
print(f"\n🎯 Melhoria alcançada: +{melhoria:.1f} pontos percentuais")

## 9. Análise com PySpark - Estatísticas de Qualidade

In [ ]:
print("📊 ANÁLISE DE QUALIDADE COM PYSPARK")
print("=" * 50)

# Análise dos dados originais
print("\n🔍 DADOS ORIGINAIS:")
print(f"Total de registros: {df_clientes_spark.count()}")
print(f"Duplicatas por ID: {df_clientes_spark.count() - df_clientes_spark.dropDuplicates(['id_cliente']).count()}")
print(f"Valores nulos em 'nome': {df_clientes_spark.filter(col('nome').isNull() | (col('nome') == '')).count()}")
print(f"Valores nulos em 'email': {df_clientes_spark.filter(col('email').isNull()).count()}")
print(f"Idades inválidas: {df_clientes_spark.filter((col('idade') < 0) | (col('idade') > 120)).count()}")

# Análise dos dados corrigidos
print("\n✅ DADOS CORRIGIDOS:")
print(f"Total de registros: {df_clientes_corrigido_spark.count()}")
print(f"Duplicatas por ID: {df_clientes_corrigido_spark.count() - df_clientes_corrigido_spark.dropDuplicates(['id_cliente']).count()}")
print(f"Valores nulos em 'nome': {df_clientes_corrigido_spark.filter(col('nome').isNull() | (col('nome') == '')).count()}")
print(f"Valores nulos em 'email': {df_clientes_corrigido_spark.filter(col('email').isNull()).count()}")
print(f"Idades inválidas: {df_clientes_corrigido_spark.filter((col('idade') < 0) | (col('idade') > 120)).count()}")

# Distribuição por status
print("\n📈 DISTRIBUIÇÃO POR STATUS (Dados Corrigidos):")
df_clientes_corrigido_spark.groupBy("status").count().orderBy("count", ascending=False).show()

# Distribuição por estado
print("\n🗺️ DISTRIBUIÇÃO POR ESTADO (Dados Corrigidos):")
df_clientes_corrigido_spark.groupBy("estado").count().orderBy("count", ascending=False).show()

## 10. Geração de Relatórios de Qualidade

In [ ]:
# Gerar Data Docs (relatórios HTML)
print("📈 GERANDO RELATÓRIOS DE QUALIDADE (DATA DOCS)")
print("=" * 50)

try:
    # Construir Data Docs
    context.build_data_docs()
    
    # Obter URLs dos relatórios
    data_docs_sites = context.get_docs_sites_urls()
    
    print("✅ Relatórios de qualidade gerados com sucesso!")
    print("\n📊 RELATÓRIOS DISPONÍVEIS:")
    
    for site_name, url in data_docs_sites.items():
        print(f"   📋 {site_name}: {url}")
        
    print("\n💡 Abra os URLs acima no navegador para visualizar os relatórios detalhados")
    
except Exception as e:
    print(f"⚠️ Erro ao gerar Data Docs: {str(e)}")

# Resumo executivo
print("\n📋 RESUMO EXECUTIVO DE QUALIDADE")
print("=" * 50)
print(f"📊 Dataset Original (PySpark):")
print(f"   - Registros: {df_clientes_spark.count()}")
print(f"   - Taxa de qualidade: {validation_result.statistics['success_percent']:.1f}%")
print(f"   - Status: {'✅ APROVADO' if validation_result.success else '❌ REPROVADO'}")

print(f"\n📊 Dataset Corrigido (PySpark):")
print(f"   - Registros: {df_clientes_corrigido_spark.count()}")
print(f"   - Taxa de qualidade: {validation_result_corrigido.statistics['success_percent']:.1f}%")
print(f"   - Status: {'✅ APROVADO' if validation_result_corrigido.success else '❌ REPROVADO'}")

melhoria = validation_result_corrigido.statistics['success_percent'] - validation_result.statistics['success_percent']
print(f"\n🎯 Melhoria alcançada: +{melhoria:.1f} pontos percentuais")
print(f"📉 Redução de registros: {df_clientes_spark.count() - df_clientes_corrigido_spark.count()} registros removidos")

## 11. Pipeline DataOps Completo com PySpark + Great Expectations

In [ ]:
def pipeline_dataops_pyspark_gx(df_spark, context):
    """
    Pipeline DataOps completo usando PySpark + Great Expectations
    """
    print("🚀 PIPELINE DATAOPS PYSPARK + GREAT EXPECTATIONS")
    print("=" * 60)
    
    pipeline_result = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "status": "EXECUTANDO",
        "etapas_concluidas": [],
        "metricas_qualidade": {},
        "spark_stats": {}
    }
    
    try:
        # Etapa 1: Análise inicial com PySpark
        print("\n📊 Etapa 1: Análise inicial com PySpark")
        pipeline_result["spark_stats"]["registros_originais"] = df_spark.count()
        pipeline_result["spark_stats"]["duplicatas"] = df_spark.count() - df_spark.dropDuplicates(['id_cliente']).count()
        pipeline_result["etapas_concluidas"].append("analise_spark")
        
        # Etapa 2: Validação com Great Expectations
        print("\n🎯 Etapa 2: Validação com Great Expectations")
        df_pandas = df_spark.toPandas()
        validator = context.sources.pandas_default.read_dataframe(df_pandas)
        
        # Aplicar expectativas
        validator.expect_column_values_to_not_be_null("id_cliente")
        validator.expect_column_values_to_be_unique("id_cliente")
        validator.expect_column_values_to_be_between("idade", min_value=0, max_value=120)
        
        validation_result = validator.validate()
        pipeline_result["metricas_qualidade"]["inicial"] = validation_result.statistics
        pipeline_result["etapas_concluidas"].append("validacao_gx")
        
        # Etapa 3: Correção com PySpark (se necessário)
        if not validation_result.success:
            print("\n🔧 Etapa 3: Correção com PySpark")
            df_corrigido = corrigir_dados_qualidade_spark(df_spark)
            pipeline_result["spark_stats"]["registros_corrigidos"] = df_corrigido.count()
            pipeline_result["etapas_concluidas"].append("correcao_spark")
            
            # Etapa 4: Re-validação
            print("\n🔍 Etapa 4: Re-validação")
            df_corrigido_pandas = df_corrigido.toPandas()
            validator_final = context.sources.pandas_default.read_dataframe(df_corrigido_pandas)
            
            # Aplicar expectativas novamente
            validator_final.expect_column_values_to_not_be_null("id_cliente")
            validator_final.expect_column_values_to_be_unique("id_cliente")
            validator_final.expect_column_values_to_be_between("idade", min_value=0, max_value=120)
            
            validation_result_final = validator_final.validate()
            pipeline_result["metricas_qualidade"]["final"] = validation_result_final.statistics
            pipeline_result["etapas_concluidas"].append("revalidacao")
            
            if validation_result_final.success:
                pipeline_result["status"] = "✅ SUCESSO"
            else:
                pipeline_result["status"] = "⚠️ SUCESSO PARCIAL"
        else:
            pipeline_result["status"] = "✅ SUCESSO"
        
        # Etapa 5: Relatórios
        print("\n📈 Etapa 5: Gerando relatórios")
        context.build_data_docs()
        pipeline_result["etapas_concluidas"].append("relatorios")
        
    except Exception as e:
        pipeline_result["status"] = "❌ ERRO"
        pipeline_result["erro"] = str(e)
    
    return pipeline_result

# Executar pipeline completo
resultado_pipeline = pipeline_dataops_pyspark_gx(df_clientes_spark, context)

print(f"\n📋 Status final do pipeline: {resultado_pipeline['status']}")
print(f"📅 Timestamp: {resultado_pipeline['timestamp']}")
print(f"✅ Etapas concluídas: {', '.join(resultado_pipeline['etapas_concluidas'])}")
print(f"📊 Estatísticas Spark: {resultado_pipeline['spark_stats']}")

## 12. Conclusões

### 🎯 O que Aprendemos com PySpark + Great Expectations

Neste laboratório, implementamos um **pipeline DataOps profissional** combinando PySpark e Great Expectations:

#### ✅ Vantagens da Combinação:
- **PySpark**: Processamento distribuído e escalável
- **Great Expectations**: Validações padronizadas e relatórios
- **Integração**: Conversão pandas para compatibilidade
- **Performance**: Correções em larga escala com Spark
- **Governança**: Controle completo da qualidade

#### ✅ 6 Dimensões Implementadas:
- **Completude**: `expect_column_values_to_not_be_null`
- **Unicidade**: `expect_column_values_to_be_unique`
- **Validade**: `expect_column_values_to_match_regex`, `expect_column_values_to_be_between`
- **Consistência**: `expect_column_value_lengths_to_equal`
- **Pontualidade**: `expect_column_values_to_be_between` (datas)
- **Acurácia**: `expect_column_values_to_not_match_regex`

### 💡 Lições Principais

> **"PySpark + Great Expectations = Pipeline DataOps escalável e profissional"**

- **Escalabilidade**: PySpark para grandes volumes
- **Padronização**: Great Expectations para validações
- **Automação**: Pipeline completo automatizado
- **Relatórios**: Data Docs profissionais

## 🧹 Limpeza do Ambiente

In [ ]:
# Finalizar Spark Session
spark.stop()
print("✅ Spark Session finalizada")
print("🎓 Laboratório PySpark + Great Expectations concluído com sucesso!")
print("\n📚 Continue explorando DataOps com PySpark e Great Expectations!")
print("\n🔗 Recursos adicionais:")
print("   - PySpark: https://spark.apache.org/docs/latest/api/python/")
print("   - Great Expectations: https://docs.greatexpectations.io/")
print("   - DataOps: https://dataops.live/")